In [1]:
from wrapper import RV32I
import numpy as np
from tqdm import tqdm

def mem_read(mem, addr):
    addr_max = len(mem)
    ret = np.uint32(0)
    for i in range(4):
        ret += (mem[addr+i] << (i * 8)) if addr + i < addr_max else 0
    return ret.item()

def mem_write(mem, addr, data):
    addr_max = len(mem)
    for i in range(4):
        if (addr+i) < addr_max:
            mem[addr+i] = (data >> (i*8)) & 0xff

In [2]:
pc = 0
mem = np.fromfile("./bin/ma_data.bin", np.uint8)
rv32i = RV32I("./rv32i/rv32i")

print("Value at mem[0x1000](0x11111111 for success): {:#10x}".format(mem_read(mem, 0x1000)))

Value at mem[0x1000](0x11111111 for success):        0x0


In [3]:
for i in tqdm(range(1000)):
    inst = mem_read(mem, pc)
    #time.sleep(0.1)
    ((wen, waddr, wdata), pc) = rv32i.forward(inst=inst, pc=pc, silent=True)
    if (wen == 1):
        #print("{:08x}: {:08x}".format(waddr, wdata))
        mem_write(mem, waddr, wdata)

print("Value at mem[0x1000](0x11111111 for success): {:#10x}".format(mem_read(mem, 0x1000)))

100%|██████████| 1000/1000 [00:01<00:00, 972.33it/s]

Value at mem[0x1000](0x11111111 for success): 0x11111111


# below is for debugging

In [3]:
for i in range(1000):
    inst = mem_read(mem, pc)
    #time.sleep(0.1)
    ((wen, waddr, wdata), pc) = rv32i.forward(inst=inst, pc=pc, silent=True)
    if (wen == 1):
        print("{:08x}: {:08x}".format(waddr, wdata))
        mem_write(mem, waddr, wdata)
    if pc == 0x50:
        print("last")
        break

last


In [12]:
print("after")
rv32i.forward(0x00000033, 0)
print("{:08x}: {:08x}".format(pc, mem_read(mem, pc)))

after
{"current_state" : "32'd64 | 32'd101 | 32'd102 | 32'd5 | 32'd104 | 32'd105 | 32'd100989949 | 32'd100992003 | 32'd4608 | 32'd109 | 32'd110 | 32'd111 | 32'd112 | 32'd113 | 32'd114 | 32'd115 | 32'd116 | 32'd117 | 32'd118 | 32'd119 | 32'd120 | 32'd121 | 32'd122 | 32'd123 | 32'd124 | 32'd125 | 32'd126 | 32'd127 | 32'd128 | 32'd129 | 32'd130 | 32'd131 # ! 1'd0 # ! 1'd0 # ! 5'd8 # ! 32'd0 # ! 32'd4615 # ! 3'd0"}
Inst: 0x00000033
PC: 0
00000060: 50731063


In [11]:
inst = mem_read(mem, pc)
    #time.sleep(0.1)
((wen, waddr, wdata), pc) = rv32i.forward(inst=inst, pc=pc, silent=False)
if (wen == 1):
    print("{:08x}: {:08x}".format(waddr, wdata))
    mem_write(mem, waddr, wdata)

{"current_state" : "32'd64 | 32'd101 | 32'd102 | 32'd5 | 32'd104 | 32'd105 | 32'd100989949 | 32'd84148994 | 32'd4608 | 32'd109 | 32'd110 | 32'd111 | 32'd112 | 32'd113 | 32'd114 | 32'd115 | 32'd116 | 32'd117 | 32'd118 | 32'd119 | 32'd120 | 32'd121 | 32'd122 | 32'd123 | 32'd124 | 32'd125 | 32'd126 | 32'd127 | 32'd128 | 32'd129 | 32'd130 | 32'd131 # ! 1'd1 # ! 1'd0 # ! 5'd7 # ! 32'd5 # ! 32'd96 # ! 3'd2"}
Inst: 0x06050403
PC: 4611


In [7]:
((wen, waddr, wdata), pc)

((1, 4096, 286331153), 428)

In [3]:
rv32i.stop()

In [ ]:
import subprocess
import re
from typing import Tuple
import json

def extract_numbers(input_string: str) -> Tuple[Tuple[int, int, int], int]:
    """
    Extract numbers from the given string in the format `((1'dAAA, (32'dBBB, 32'dCCC)), 32'dDDD)`
    and return them as a tuple of integers.

    Args:
        input_string (str): The input string containing the numbers.

    Returns:
        Tuple[Tuple[int, int, int], int]: A tuple in the format ((AAA, BBB, CCC), DDD).
    """
    # Regular expression to extract numbers after "d"
    pattern = r"\d+'d(\d+)"
    
    # Find all matches of numbers in the input string
    matches = re.findall(pattern, input_string)
    
    # Convert matches to integers
    numbers = list(map(int, matches))
    
    # Create and return the desired tuple structure
    return ((numbers[0], numbers[1], numbers[2]), numbers[3])

def mk_fwd(fn_process: subprocess):
    def fwd(inst:int, pc:int) -> Tuple[Tuple[int, int, int], int]:
        inst_str = "{}\n".format(inst)
        pc_str = "{}\n".format(pc)

        current_state = rv32i.stdout.readline().rstrip("\n")
        req_inst = rv32i.stdout.readline().rstrip("\n")

        print(req_inst + inst_str, end="")

        rv32i.stdin.write(inst_str)
        rv32i.stdin.flush()

        req_pc = rv32i.stdout.readline().rstrip("\n")

        print(req_pc + pc_str, end="")

        rv32i.stdin.write(pc_str)
        rv32i.stdin.flush()

        out_str = json.loads(rv32i.stdout.readline())["output"]
        #out_str
        return extract_numbers(out_str)
    
    return fwd


rv32i = subprocess.Popen('./rv32i/rv32i', stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

fwd = mk_fwd(rv32i)


In [2]:
print(fwd(100, 4))

Inst: 100
PC: 4
((1'd0, (32'd4, 32'd0)), 32'd8)((0, 4, 0), 8)


In [3]:
fwd(200, 8)

Inst: 200
PC: 8
((1'd0, (32'd8, 32'd0)), 32'd12)

((0, 8, 0), 12)